# Scrape NYT

This notebook contains code to scrape The New York Times.

Load dependencies and data.

In [90]:
import pandas as pd
from ast import literal_eval
import time
import json

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import RegexpTokenizer

import configparser
configs = configparser.ConfigParser()
configs.read('../../config.ini')

['../../config.ini']

In [2]:
data = pd.read_csv('data/all.csv')
data['keywords'] = data['keywords'].apply(literal_eval)
data['date'] = pd.to_datetime(data['date']) 
data

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,False,False,False,False,False,False,False
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,False,False,False,False,False,False,False
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"[Homeless Persons, HOTELS AND MOTELS, Children...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,False,False,False,False,False,False,False
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,[Company Reports],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,False,False,False,False,False,False,False
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"[Space and Astronomy, United States Defense an...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17340,Your Money; Claiming a Pet As a Deduction,1981-03-28,article,News,Financial Desk,Business Day,"[ANIMALS, Taxation, Income Tax, Handicapped]",https://www.nytimes.com/1981/03/28/business/yo...,nyt://article/8aa2aceb-e543-5691-b4cc-572cfada...,By Elizabeth M. Fowler,False,False,False,True,False,False,False,True
17341,"Your Typical Crowded, Swinging, Silent Bar Scene",1994-10-30,article,News,The City Weekly Desk,New York,"[Deafness, Bars]",https://www.nytimes.com/1994/10/30/nyregion/ne...,nyt://article/1aab92b9-5b05-50b7-bf7a-2b8cef19...,By Jennifer Kingson Bloom,False,False,False,True,False,False,False,True
17342,"‘Fargo’ Recap: Dead Dogs, Spiders and Pestilence",2014-04-30,article,News,Culture,Arts,[],https://artsbeat.blogs.nytimes.com/2014/04/29/...,nyt://article/3a6161c6-023a-5968-a28e-0ea2ecb6...,By Kate Phillips,False,False,False,True,False,False,False,True
17343,‘Singing’ With Their Hands,2012-02-11,article,News,Styles,Fashion & Style,"[Video Recordings and Downloads, Music, Sign L...",https://www.nytimes.com/2012/02/12/fashion/sin...,nyt://article/0918d106-bd33-59fe-a100-cd3f9a23...,By Austin Considine,False,False,False,True,False,False,False,True


Load phrases.

In [3]:
with open('phrases.txt', 'r') as infile:
    phrases = json.load(infile)
    
phrases

{'deaf_and_dumb': ['deaf and dumb', 'deaf dumb'],
 'deaf_mute': ['deaf mute', 'deaf and mute', 'mute deaf', 'mute and deaf'],
 'fell_on_deaf_ears': ['fell on deaf ears',
  'fall on deaf ears',
  'falls on deaf ears',
  'fall on a deaf ear',
  'falling on deaf ears',
  'falling on a deaf ear',
  'turn a deaf ear',
  'turned deaf ears',
  'turned a deaf ear',
  'turning deaf ears',
  'turning a deaf ear'],
 'hearing_impaired': ['hearing impaired', 'hearing impairment'],
 'tone_deaf': ['tone deaf'],
 'deaf_as_a_post': ['deaf as a post'],
 'stone_deaf': ['stone deaf'],
 'deaf': ['deaf']}

Extract data subset for only the True/False columns. We use this when scraping to see which phrases each article has.

In [224]:
data_subset = data.loc[:, 'deaf_and_dumb':'deaf']
column_names = data_subset.columns.values.astype(str)
phrases_for_each_article = [column_names[i].tolist() for i in data_subset.values]

## Scrape sentences into sentences vector/Series.

Define functions to log into the New York Times site, browse to each article, and scrape matching sentences.

In [221]:
def connect_to_nyt():
    # Open browser and navigate to NYT
    browser = webdriver.Chrome(executable_path='../../chromedriver')
    browser.get('https://nyt.com')

    # Bring up login portal
    wait = WebDriverWait(browser, 15)
    login_button_XPATH = '//button[@data-testid="login-button"]'
    login_button_present = EC.presence_of_element_located((By.XPATH, login_button_XPATH))
    login_button = wait.until(login_button_present)
    login_button.click()

    # Log in
    fields_present = EC.presence_of_element_located((By.ID, 'username'))
    wait.until(fields_present).send_keys(configs['NYT']['EMAIL'])
    browser.find_element_by_id('password').send_keys(configs['NYT']['PASSWORD'])
    browser.find_element_by_xpath(login_button_XPATH).click()
    
    return browser
    
    
def scrape_sentences(article):
    time.sleep(4)
    article_id = article['id']
    url = article['url']
    
    # Navigate to page
    browser.get(article['url'])
    
    status = ['Full text is unavailable for this digitized archive article.',
              'Page Not Found',
              'Server Error',
              'Success',
              'Failed']
    
    if status[0] in browser.page_source: # time machine
        print(status[0]) 
        return status[0], None
    elif status[1] in browser.page_source: # page not found
        print(status[1])
        return status[1], None
    elif status[2] in browser.page_source: # server error
        print(status[2])
        return status[2], None
    else:
        # Get all sentences
        paragraphs = browser.find_elements_by_tag_name('p')
        text = ' '.join([p.text for p in paragraphs])
        sentences = [x for x in tokenizer.tokenize(text)] # break into sentences
        sentences_cleaned = [' '.join(RegexpTokenizer(r'\w+').tokenize(x)).lower() for x in sentences] # remove punctuation, lowercase
         
        # Get all varieties of the phrases for this article
        phrases_for_this_article = phrases_for_each_article[article.name]
        phrase_varieties = ['deaf'] # add by default, also because we made the 'deaf' column to exclude other 'deaf' terms
        for phrase in phrases_for_this_article:
            phrase_varieties += [x for x in phrases[phrase]]
        
        # Get sentences that contain a phrase variety
        matches = []
        for p in phrase_varieties:
            matches += ([sentences[i] for i, x in enumerate(sentences_cleaned) if (p in x and sentences[i] not in matches)])
        
        if len(sentences) >= 1: 
            print(status[3] + ': ' + str(len(matches)) + ' sentence(s) found.') # success
            return status[3], matches
        else:
            print(status[4]) # failed
            return status[4], None

Scrape sentences. Each row will contain either of 3 values:
- `page not found` — if the page was not found
- `server error` — if there was a server error
- `time machine` — if there is no fulltext available, and the article needs to be viewed through Time Machine
- `[]` - array of sentence(s)

In [49]:
browser = connect_to_nyt()

In [226]:
statuses, sentences = zip(*data.iloc[0:10].apply(scrape_sentences, axis=1))

Full text is unavailable for this digitized archive article.
Success: 1 sentence(s) found.
Success: 1 sentence(s) found.
Success: 1 sentence(s) found.
Success: 1 sentence(s) found.
Full text is unavailable for this digitized archive article.
Full text is unavailable for this digitized archive article.
Full text is unavailable for this digitized archive article.
Success: 1 sentence(s) found.
Success: 1 sentence(s) found.


**To do:**
- Find regex for each phrase, and when article contains the phrase, use these regexes to target sentences instead of the current approach `if term in sentence` which would get lengthy for some phrases like "fall on deaf ears" which have multiple varieties. But again, the regex could get complex as well. Maybe use something like the queries format in query_nyt.ipynb.
    - if article contains multiple phrases, e.g. has two True columns in `data`, use something like contains(regex) OR contains(regex)
- Include headline in sentences array for each article, if headline contains the phrase
- For the articles that are unavailable as fulltexts, I can collect their URLs based on sentences==None then maybe automatically direct browser to the TimeMachine, then I will manually read it then [type the sentence into Jupyter Notebook using user input](https://stackoverflow.com/questions/34968112/how-to-give-jupyter-cell-standard-input-in-python) and then the program will add this to sentences cell for that article row, move on to the next row that needs it, then redirect browser automatically, and so on. 

In [229]:
sentences

(None,
 ['"The international community has been totally deaf and dumb on the situation between Sudan and Chad," Mr. Deby said, declaring that "enough is enough."'],
 ["'Like I Am in a Coffin' Her first husband, Ms. Bruce said, left her when the doctor announced their son was deaf and dumb."],
 ['"You\'d have to be blind, deaf and dumb to not have known that oil prices went up," said Bernard Sosnick, a retail analyst at Oppenheimer & Company.'],
 ['“We could be deaf, dumb and blind within seconds,” Mr. Cooper, who sits on the House Armed Services Committee with Mr. Rogers, said in February at a space forum at the Center for Strategic and International Studies.'],
 None,
 None,
 None,
 ["He struggled to raise himself to his feet; he undermined John Gray's already tottering reason with a succession of graceful but complex signs drawn from the deaf and dumb language; then he began to rail at Gray, in a peculiarly barbarous foreign tongue for idling there and looking stupid when he ought to